# BULKING NOTEBOOK 

In [21]:
import numpy as np
import pandas as pd
import sqlite3
import datetime
import re

### STEPS FOR DATABASING
1. read in data
1.1 connect to db
2. create data base and check that it does not exist
2.1 create raw table
3. clean and transform data function
3.1 data times and ints and no strings in int catagories
4. create silver/transformed table
5. move data to silver table.

In [22]:
dtype = {
    'Date': str,
    'weigh(lbs)': float,
    'workout_days': int,
    'missed_meals': int,
    'protein': int,
    'creatine': int,
}

In [23]:
path = '/Users/geraldlittlejr/Documents/vs_files/bulking/bulking_file.txt'
df = pd.read_csv(path)
df

,Date,weight(lbs),workout_days,missed_meals,protein,creatine
0,Nov202023,200.2,4.0,3,1.0,1.0
1,Nov272023,202.2,5.0,1,1.0,1.0
2,Dec042023,206.6,4.5,0,0.0,1.0
3,Dec112023,207.6,4.5,2,1.0,1.0
4,Dec182023,208.6,2.0,6,0.0,0.0
5,Dec262023,204.8,1.0,7,0.0,0.0
6,JAN022024,207.5,3.0,5,1.0,1.0
7,Jan092024,204.8,3.0,5,1.0,1.0
8,JAN162024,206.5,1.0,3(?),1.0,1.0
9,Jan222024,211.4,5.0,1,1.0,1.0


In [24]:
# Create db instance, connection and cursor to db instance
name = "fitness_tracker_database.db"
con = sqlite3.connect(name)
cur = con.cursor()
bronze_table = "bronze_table"
silver_table = "silver_table"
gold_table = "gold_table"

In [25]:
# Create table and insert data from pandas dataframe
try:
    df.to_sql(bronze_table, 
              con, 
              if_exists='replace', 
              index=False,
              dtype={
                  'Date': 'TEXT',
                  'weight(lbs)': 'TEXT',
                  'workout_days': 'TEXT',
                  'missed_meals': 'TEXT',
                  'protein': 'TEXT',
                  'creatine': 'TEXT',
})

except Exception as e:
    print(f'There was an error with table creation: ', e)
else:
    print(f'Table created successfully')
    
    # CHECC THAT A TABLE WAS CREATED
    res = cur.execute(f"SELECT name FROM sqlite_master").fetchall()
    print(res)

Table created successfully
[('silver_table',), ('bronze_table',)]


In [26]:
cur.execute(f"SELECT * FROM bronze_table")
rows = cur.fetchall()
for row in rows:
    print(row)


('Nov202023', '200.2', '4.0', '3', '1.0', '1.0')
('Nov272023', '202.2', '5.0', '1', '1.0', '1.0')
('Dec042023', '206.6', '4.5', '0', '0.0', '1.0')
('Dec112023', '207.6', '4.5', '2', '1.0', '1.0')
('Dec182023', '208.6', '2.0', '6', '0.0', '0.0')
('Dec262023', '204.8', '1.0', '7', '0.0', '0.0')
('JAN022024', '207.5', '3.0', '5', '1.0', '1.0')
('Jan092024', '204.8', '3.0', '5', '1.0', '1.0')
('JAN162024', '206.5', '1.0', '3(?)', '1.0', '1.0')
('Jan222024', '211.4', '5.0', '1', '1.0', '1.0')
('Jan292024', '210.4', '1.0', '3', '0.0', '0.0')
('Feb05', '210', '1.0', '4', '1.0', '1.0')
('Feb122024', '212.6', '4.0', '2', '1.0', '1.0')
('Feb192024', '212.6', '1.0', '6', '0.0', '0.0')
('Feb262024', '211.8', '1.0', '2', '0.0', '0.0')
('Mar032024', '215.6(?)', '0.0', '3', '0.0', '0.0')
('Mar112024', '216.8', '0.0', '0', '0.0', '0.0')
('Mar252024', '215', '0.0', '2', '0.0', '0.0')
('1APR2024', '215.6', '1.0', '3', '0.0', '0.0')
('7APR2024', '217.8', '0.0', '4', '0.0', '0.0')
('14APR2024', '217.5', '

In [27]:
# CHECC FOR NON EXISTANT TABLE
res = cur.execute(f"SELECT name FROM sqlite_master WHERE name='foo'")
res.fetchone() is None

True

In [28]:
# Table Transformations
def clean_numerical(col):
    import re
    # Remove any non-numeric characters using regular expressions
    return re.sub(r'[^\d.]+', '', col)

def add_default_year(date_str, default_year=2024):
    if len(date_str) == 5:
        return date_str + default_year
    return date_str

def parse_date(date):
    # Convert to string if not already
    date = str(date)
    
    # Handle mmmdd format (e.g., 'Mar05', 'Dec15') by appending 2024
    if len(date) == 5 and date[0:3].isalpha():
        date += '2024'
    
    # Define the possible formats
    formats = ['%m%d%Y', '%d%b%Y', '%b%d%Y']
    
    # Try each format
    for fmt in formats:
        try:
            return pd.to_datetime(date, format=fmt)
        except ValueError:
            continue

def table_transform(df):
    df.fillna('-1', inplace=True)
    df['Date'] = df['Date'].apply(parse_date)
    df['weight(lbs)'] = df['weight(lbs)'].apply(clean_numerical).astype(float)
    df['workout_days'] = df['workout_days'].apply(clean_numerical).astype(float)
    df['missed_meals'] = df['missed_meals'].apply(clean_numerical).astype(int)
    df['protein'] = df['protein'].apply(clean_numerical).astype(float)
    df['creatine'] = df['creatine'].apply(clean_numerical).astype(float)



    return df

In [29]:
# Read data from the database and transform it
try:
    silver_df = pd.read_sql_query(f"SELECT * FROM {bronze_table}", con)
    # print(silver_df)
    transform_df = table_transform(silver_df)
    
except AttributeError as e:
    print('There was an error with the transformation function:', e)
except sqlite3.OperationalError as e:
    print('There was an error with the SQLite operation:', e)
except Exception as e:
    print('An unexpected error occurred:', e)
finally:
    print(transform_df)

         Date  weight(lbs)  workout_days  missed_meals  protein  creatine
0  2023-11-20        200.2           4.0             3      1.0       1.0
1  2023-11-27        202.2           5.0             1      1.0       1.0
2  2023-12-04        206.6           4.5             0      0.0       1.0
3  2023-12-11        207.6           4.5             2      1.0       1.0
4  2023-12-18        208.6           2.0             6      0.0       0.0
5  2023-12-26        204.8           1.0             7      0.0       0.0
6  2024-01-02        207.5           3.0             5      1.0       1.0
7  2024-01-09        204.8           3.0             5      1.0       1.0
8  2024-01-16        206.5           1.0             3      1.0       1.0
9  2024-01-22        211.4           5.0             1      1.0       1.0
10 2024-01-29        210.4           1.0             3      0.0       0.0
11 2024-02-05        210.0           1.0             4      1.0       1.0
12 2024-02-12        212.6           4

In [30]:
# Create table and insert data from pandas dataframe
try:
    transform_df.to_sql(silver_table, 
        con, 
        if_exists='replace', 
        index=False,
        dtype={
            'Date': 'TEXT',
            'weight(lbs)': 'REAL',
            'workout_days': 'REAL',
            'missed_meals': 'INTEGER',
            'protein': 'REAL',
            'creatine': 'REAL',
})

except Exception as e:
    print(f'There was an error with table creation: ', e)
else:
    print(f'Table created successfully')
    
    # CHECC THAT A TABLE WAS CREATED
    res = cur.execute(f"SELECT name FROM sqlite_master").fetchall()
    print(res)

Table created successfully
[('bronze_table',), ('silver_table',)]
